# 🖊️ Pen Plotter — Clean Quickstart (GRBL, Safe)

A modular, beginner-friendly workflow for controlling a GRBL-based pen plotter with Python.

- Connect and configure your device
- Manually home the arm (move to Arduino for origin)
- Calibrate pen contact
- Plot a custom SVG
- Safely close the connection

All logic is organized in `penplot_helper.py`. Follow the steps below!

## 🏠 Manual Homing

Move the pen arm gently towards the Arduino to set the origin (x=0, y=0).
This step ensures accurate positioning before plotting.

## ⚙️ Initialization

Set up your pen plotter and connect to GRBL below.
Make sure your device is on the correct port.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from penplot_helper import Config, GRBL
import numpy as np
import matplotlib.pyplot as plt

# Connect
cfg = Config(
    port="/dev/tty.usbserial-A50285BI",
    x_max=300.0, y_max=245.0,
    s_down=90, s_up=40,
)
grbl = GRBL(cfg).connect()

# Warm up status and print it
grbl.ensure_wpos()
grbl.status()


{'raw': '<Idle|MPos:0.000,0.000,0.000|Bf:15,128|FS:0,0|Ov:100,100,100>',
 'state': 'Idle',
 'wpos': (0.0, 0.0, 0.0)}

## Test rectangle sweep to set paper size

In [3]:
# Define bed and do a quick perimeter check (pen up)
grbl.pen_up(step=0.1)
#grbl.set_bed(300.0, 225.0)
grbl.sweep_rect(cfg.x_max, cfg.y_max)

# Go to center (absolute work coords)
grbl.goto_abs(cfg.x_max/2.0, cfg.y_max/2.0)
grbl.goto_abs(  0.0,   0.0)

grbl.pen_down(step=0.1)

Goto absolute: (150.000, 122.500)
Goto absolute: (0.000, 0.000)


In [3]:
grbl.status()

{'raw': '<Idle|MPos:0.000,0.000,0.000|Bf:15,128|FS:0,0>',
 'state': 'Idle',
 'wpos': (0.0, 0.0, 0.0)}

## 📐 Select Project Area

Interactively choose the bottom left and top right corners of your project area. The surface compensation will use these bounds for calibration.

In [4]:
from penplot_widgets import show_area_and_compensation_widget
get_area_and_comp = show_area_and_compensation_widget(grbl)  # Interactively set area and compensation (motor updates on slider change)

## Setup conveniences

In [5]:
import penplot_widgets as ppw
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
saved = getattr(ppw, "_PPW_STATE", {}).get(key, None)
saved


{'corners': {'BL': {'x': 0.0, 'y': 0.0, 'h': 0.32},
  'BR': {'x': 220.0, 'y': 0.0, 'h': 0.3},
  'TL': {'x': 0.0, 'y': 158.6, 'h': 0.38},
  'TR': {'x': 220.0, 'y': 158.6, 'h': 0.54}},
 'pots': []}

### Test Pattern

In [8]:
from pattern import Pattern, Renderer, Polyline, Circle, Line
import math, random


# Minimal pattern: 2 circles + 1 line + 1 polyline
p = Pattern()
p.add(
    Circle((30.0, 30.0), 14.0, pen_pressure=-0.1, start_deg=0,   sweep_deg=360, feed_draw=5000, pen_id=0),
    Circle((70.0, 50.0), 18.0, pen_pressure=-0.05, start_deg=180, sweep_deg=180, feed_draw=5000, pen_id=1),  
    Line((20.0, 80.0), (100.0, 80.0), pen_pressure=-0.05, feed_draw=10000),
    Polyline([(20.0, 20.0), (50.0, 25.0), (80.0, 22.0)], pen_pressure=-0.0, feed_draw=1000),
)

# Renderer (your params)
r = Renderer(grbl,
             z_mode="centroid",
             z_threshold=0.1,
             settle_down_s=0.04,
             settle_up_s=0.04,
             z_step=0.02,
             z_step_delay=0.00,
             flush_every=300,
             feed_travel=15000,
             lift_delta=0.8)

r.plot(p)

# r.run(p,
#       pen_filter = [0,1], 
#       start_xy=(0.0, 0.0),  # for nn optimization
#       optimize='nn',
#       combine={'join_tol_mm': 0.1},
#       resample={'max_dev_mm': 0.1, 'max_seg_mm': None})


### Truchet tiles

In [6]:
from pattern import Pattern, Renderer, Polyline, Circle, Line
import random, math

def offsets(tile, bands, margin, bias_p=2.0):
    if bands <= 0: return []
    if bands == 1: return [tile*0.5]
    c = (bands-1)/2
    gaps = [1/((1+abs(i+0.5-c))**bias_p) for i in range(bands-1)]
    span = tile-2*margin
    s, pos = margin, [margin]
    k = span/sum(gaps)
    for g in gaps:
        s += g*k
        pos.append(s)
    return [round(max(margin, min(tile-margin, x)), 6) for x in pos]

def pen_for_band(i, pen_main, pen_alt, highlight_band=None):
    return pen_alt if (highlight_band is not None and i == highlight_band) else pen_main

def add_vertical_bundle(p, x0, y0, T, pos, feed, press, pen_main, pen_alt, highlight_band):
    for i,x in enumerate(pos):
        p.add(Line((x0+x,y0),(x0+x,y0+T), pen_id=pen_for_band(i,pen_main,pen_alt,highlight_band),
                   pen_pressure=press, feed_draw=feed))

def add_horizontal_bundle_cut_block(p, x0, y0, T, pos_y, bundle_xs, feed, press, pen_main, pen_alt, highlight_band, margin=0.0):
    # cut from first to last vertical across the whole bundle (plus optional margin)
    if not bundle_xs: 
        xs = (x0, x0+T)
    else:
        xs = (x0+min(bundle_xs)-margin, x0+max(bundle_xs)+margin)
    for i,y in enumerate(pos_y):
        yA = y0+y
        leftA, rightA = x0, x0+T
        # left segment
        if xs[0]-leftA > 1e-3:
            p.add(Line((leftA,yA),(xs[0],yA),
                       pen_id=pen_for_band(i,pen_main,pen_alt,highlight_band),
                       pen_pressure=press, feed_draw=feed))
        # right segment
        if rightA-xs[1] > 1e-3:
            p.add(Line((xs[1],yA),(rightA,yA),
                       pen_id=pen_for_band(i,pen_main,pen_alt,highlight_band),
                       pen_pressure=press, feed_draw=feed))

def add_curves_both_with_priority(p, x0, y0, T, pos, feed, press, pen_hi, pen_lo, highlight_band,
                                  hi_family="NE_SW", cut_window=(30.0,60.0)):
    # helper to draw quarter arcs, optionally cut by angular window (degrees) relative to the quarter start
    def qarc(cx, cy, start_deg, r, pen, cut=None):
        if cut is None:
            p.add(Circle((cx,cy), r, start_deg=start_deg, sweep_deg=90, pen_id=pen,
                         pen_pressure=press, feed_draw=feed))
        else:
            a0,a1 = start_deg+cut[0], start_deg+cut[1]
            if a0-start_deg > 0.1:
                p.add(Circle((cx,cy), r, start_deg=start_deg, sweep_deg=a0-start_deg, pen_id=pen,
                             pen_pressure=press, feed_draw=feed))
            if start_deg+90 - a1 > 0.1:
                p.add(Circle((cx,cy), r, start_deg=a1, sweep_deg=start_deg+90-a1, pen_id=pen,
                             pen_pressure=press, feed_draw=feed))
    # map families to corners and starts
    fams = {
        "NE_SW": [((x0+T,y0+T),180), ((x0,y0),0)],   # NE and SW
        "NW_SE": [((x0,y0+T),270),   ((x0+T,y0),90)] # NW and SE
    }
    hi, lo = hi_family, ("NW_SE" if hi_family=="NE_SW" else "NE_SW")
    # draw high family full arcs
    for i,r in enumerate(pos):
        if r <= 0 or r > T: continue
        pen_i = pen_for_band(i, pen_hi, pen_lo, highlight_band)
        for (cx,cy), s in fams[hi]:
            qarc(cx,cy,s,r,pen_i,None)
    # draw low family cut by shared window
    for i,r in enumerate(pos):
        if r <= 0 or r > T: continue
        pen_i = pen_for_band(i, pen_lo, pen_hi, highlight_band)
        for (cx,cy), s in fams[lo]:
            qarc(cx,cy,s,r,pen_i,cut_window)

def make_tiles_rect(origin=(0.0,0.0), size_x=180.0, size_y=120.0, tile=16.0,
                    bands=5, margin=1.6, bias_p=2.0, seed=12345,
                    feed_draw=2400, pen_pressure=-0.05,
                    pen_ids=(0,1), highlight_band=None,
                    straight_ratio=0.1, cut_margin=0.0,
                    curve_priority="NE_SW", curve_cut_window=(30.0,60.0)):
    rng = random.Random(seed)
    nx, ny = max(int(size_x//tile),1), max(int(size_y//tile),1)
    ox, oy = origin
    pos = offsets(tile, bands, margin, bias_p)
    p = Pattern()
    penA, penB = pen_ids
    for j in range(ny):
        for i in range(nx):
            x0, y0 = ox+i*tile, oy+j*tile
            if rng.random() < straight_ratio:
                # STRAIGHT: vertical bundle (full), horizontals cut as one block spanning first..last vertical
                add_vertical_bundle(p, x0, y0, tile, pos, feed_draw, pen_pressure, penA, penB, highlight_band)
                add_horizontal_bundle_cut_block(p, x0, y0, tile, pos, pos, feed_draw, pen_pressure,
                                                penB, penA, highlight_band, margin=cut_margin)
            else:
                # CURVES_BOTH: both families present; low family cut by one wide window so crossings are entirely removed
                add_curves_both_with_priority(p, x0, y0, tile, pos, feed_draw, pen_pressure,
                                              penA, penB, highlight_band, hi_family=curve_priority,
                                              cut_window=curve_cut_window)
    return p


art = make_tiles_rect(
    origin=(50.0,50.0), size_x=100.0, size_y=100.0, tile=12.5,
    bands=3, margin=1.5, bias_p=2.2, seed=20250923,
    feed_draw=2000, pen_pressure=-0.06,
    pen_ids=(1,2),            # two colors
    highlight_band=2,         # 0-based: band index 2 (the 3rd line) always uses pen 2
    straight_ratio=0.4,       # mix straight vs curves
    cut_margin=0.4,           # extend the removed block slightly beyond first/last crossing
    curve_priority="NE_SW",
    curve_cut_window=(28.0,62.0)  # wide removal across the entire crossing region
)

r = Renderer(grbl, z_mode="centroid", z_threshold=0.1,
             settle_down_s=0.04, settle_up_s=0.04,
             z_step=0.02, z_step_delay=0.00,
             flush_every=400, feed_travel=5000, lift_delta=0.4)
r.plot(art)


In [ ]:
# r.run(art,
#       pen_filter = [1], 
#       start_xy=(0.0, 0.0),  # for nn optimization
#       optimize='nn',
#       combine={'join_tol_mm': 0.1},
#       resample={'max_dev_mm': 0.1, 'max_seg_mm': None})

### Plantes horizontal lines

In [9]:
# --- Gravity-flow “scanlines”: 120 horizontal rays entering from the left, deflected by hard-coded planets ---
# Updated for the new API: z/settling behavior lives on the Renderer (not per-line).
# Prereqs: cfg, grbl, Pattern/Polyline/Renderer, and a saved rectangle via the widget.

import math, numpy as np, penplot_widgets as ppw
from pattern import Pattern, Polyline, Renderer

# 1) Load calibrated rectangle (+ program Z surface map)
key   = (round(cfg.x_max, 3), round(cfg.y_max, 3))
state = (getattr(ppw, "_PPW_STATE", {}) or {}).get(key) or {}
BL = state.get("corners", {}).get("BL"); TR = state.get("corners", {}).get("TR")
assert BL and TR, "Calibration rectangle not found. Open the widget and Save Settings."
x0, y0, x1, y1 = float(BL["x"]), float(BL["y"]), float(TR["x"]), float(TR["y"])

# 2) Parameters
W, H = (x1 - x0), (y1 - y0)
circle_samples     = 120

n_rays             = 160         # number of horizontal rays
launch_inset_mm    = 0.0         # inset from left edge
y_inset_mm         = 0.0         # inset from top/bottom for seeding
start_jitter_mm    = 0.0         # optional vertical jitter (0 = evenly spaced)

# Integration (smooth arclength + ramped speed)
step_mm     = 0.45
substeps    = 4
v0_mm       = 0.1
cruise_mm   = 0.1
ramp_steps  = 50
max_steps   = 1200

soft_mm     = 0.0#2.0
G_eff       = 0.002
accel_clip  = 100000
hit_margin_mm = 0.0

# Motion / pen (per-path feed stays on the polyline; Z strategy lives on Renderer)
feed_draw   = 1500

# 3) Hard-coded planets (fx, fy, fr) w.r.t. the calibrated rectangle
#    fx, fy in [0..1], fr is radius as a fraction of min(W,H).
#    All planets are placed on the RIGHT HALF (fx >= 0.55). Edit this list as you like.
PLANETS_FRAC = [
    (0.58, 0.02, 0.035),
    (0.40, 0.35, 0.020),
    (0.69, 0.24, 0.012),
    (0.73, 0.12, 0.028),
    (0.77, 0.30, 0.045),
    (0.82, 0.18, 0.018),
    (0.86, 0.42, 0.024),
    (0.90, 0.25, 0.010),
    (0.93, 0.12, 0.016),
    (0.42, 0.60, 0.030),
    (0.66, 0.76, 0.020),
    (0.74, 0.68, 0.014),
    (0.80, 0.95, 0.038),
    (0.88, 0.70, 0.022),
    (0.94, 0.95, 0.012),
]

# 4) Convert planets to absolute coords (cx, cy, r, mass)
R0 = min(W, H)
planets = []
for fx, fy, fr in PLANETS_FRAC:
    cx = x0 + fx * W
    cy = y0 + fy * H
    r  = fr * R0
    m  = r * r
    planets.append((cx, cy, r, m))

# 5) Utilities
def circle_poly(cx, cy, r, n=circle_samples):
    return [(cx + r*math.cos(2*math.pi*i/n), cy + r*math.sin(2*math.pi*i/n)) for i in range(n+1)]

def gravity_accel(x, y):
    ax = ay = 0.0
    for (cx, cy, r, m) in planets:
        dx, dy = (cx - x), (cy - y)
        d2 = dx*dx + dy*dy + soft_mm*soft_mm
        inv_d = 1.0 / math.sqrt(d2)
        inv_d3 = inv_d * inv_d * inv_d
        a = G_eff * m * inv_d3
        ax += a * dx
        ay += a * dy
    mag = math.hypot(ax, ay)
    if mag > accel_clip:
        ax *= accel_clip / mag
        ay *= accel_clip / mag
    return ax, ay

def inside_rect(x, y): return (x0 <= x <= x1) and (y0 <= y <= y1)
def hits_planet(x, y):
    for (cx, cy, r, _) in planets:
        if math.hypot(x - cx, y - cy) <= (r + hit_margin_mm):
            return True
    return False

# 6) Build pattern (planet outlines + rays)
pat = Pattern()

# Planet outlines (optional; comment out if you don't want circles drawn)
for (cx, cy, r, _) in planets:
    pat.add(Polyline(pts=circle_poly(cx, cy, r), pen_id=0, feed_draw=1000))

# Launch horizontal rays from left edge
x_start = x0 + launch_inset_mm
ys = np.linspace(y0 + y_inset_mm, y1 - y_inset_mm, n_rays)
if start_jitter_mm > 0:
    ys = np.array([float(y + np.random.uniform(-start_jitter_mm, start_jitter_mm)) for y in ys])

for y_start in ys:
    if hits_planet(x_start, y_start):
        continue

    x, y = x_start, y_start
    vx, vy = v0_mm, 0.0
    ax, ay = gravity_accel(x, y)
    pts = [(x, y)]

    for step in range(max_steps):
        v_target = v0_mm + (cruise_mm - v0_mm) * min(1.0, step / float(ramp_steps))
        ds  = step_mm
        dsm = ds / float(substeps)
        broke = False

        for _ in range(substeps):
            dt = dsm / max(1e-9, v_target)

            # half-kick
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt
            # enforce target speed
            vm = math.hypot(vx, vy) or 1.0
            vx *= (v_target / vm)
            vy *= (v_target / vm)
            # drift
            x += vx * dt
            y += vy * dt

            if (not inside_rect(x, y)) or hits_planet(x, y):
                broke = True
                break

            # update accel + half-kick
            ax, ay = gravity_accel(x, y)
            vx += 0.5 * ax * dt
            vy += 0.5 * ay * dt

        if broke:
            break
        pts.append((x, y))

    if len(pts) >= 2:
        pat.add(Polyline(pts=pts, pen_id=0, feed_draw=feed_draw))

# 7) Render (all z/settle behavior lives on the Renderer)
r = Renderer(
    grbl,
    z_mode="start",
    z_threshold=0.0,
    settle_down_s=0.15,
    settle_up_s=0.15,
    z_step=0.02,
    z_step_delay=0.00,
    flush_every=400,
    feed_travel=3000,
    lift_delta=0.5,
)
r.plot(pat)


In [10]:
r.run(pat,
      pen_filter = [0], 
      start_xy=(0.0, 0.0),  # for nn optimization
      #optimize='nn',
      #combine={'join_tol_mm': 0.1},
      resample={'max_dev_mm': 0.1, 'max_seg_mm': None})

Resample: 175 polylines, points 61622 -> 2948 (x0.05), max_dev=0.1, max_seg=None.


TimeoutError: GRBL did not become IDLE in time.

## Run experiments

## 🧹 Cleanup

Return the pen to origin and close the serial connection when finished.

In [ ]:
grbl.goto_abs(0,0)
grbl.pen_down(step=0.01)
grbl.close()
print('Serial closed.')